In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from attr import define, field
from typing import List, Union, ClassVar, Optional, TypeVar
from pathlib import Path
from pathlib import Path
import glob
import json

from typing import TypeVar
import glob
import parse
from concurrent.futures import ThreadPoolExecutor, as_completed
from datasetconfig import DatasetConfig
from dataset import Dataset, DatasetType
from seqentry import SeqEntry

from bedtoolsexecutor import BedtoolsExecutor
from chipseqprotocol import ChIPSeqIrisProtocol
from benchmarkconfig import BenchmarkConfig
from chipseqdataset import ChIPSeqDataset

In [33]:
AVAILABLE_PROCS: int = 20
BEDTOOLS_PATH: Path = Path("/home_local/dpenzar/bedtools2/bin")
BENCHMARK_CONFIG = Path("/home_local/dpenzar/chipseq_config.json")
CHS_ROOT: Path = Path("/home_local/vorontsovie/greco-data/release_7a.2021-10-14/full/CHS/Val_intervals")
DS_DIR = Path('.ChIPSeqDatasets2')
MODELS_PATH = Path("/home_local/dpenzar/models.json")
ChIPSeqLOGS = Path(".chipseq_logs")


In [5]:
def peakfls2configs(files: list[Path], 
                    name_parser= parse.compile("{tf_name}.{fl}@{exp_type}@{name}@Peaks.{unique_tag}.{dt_type}.peaks")) -> list[DatasetConfig]:
    all_records = []
    for val_fl in files:
        val_fl = val_fl.absolute()
        data = name_parser.parse(val_fl.name)
        if data is None or isinstance(data, parse.Match):
            raise Exception("Wrong peakfile name format")
        dt = {
            "name": data["name"],
            "exp_type": "ChIPSeq",
            "tf": data['tf_name'],
            "ds_type": data['dt_type'],
            "path": val_fl,
            "curation_status": "accepted",
            "protocol": "iris",
            "metainfo": {}
        }
        all_records.append(DatasetConfig.from_dict(dt))
    return all_records

In [6]:
BedtoolsExecutor.set_defaull_executor(BEDTOOLS_PATH)

In [7]:
val_files = [Path(x) for x in glob.glob(str(CHS_ROOT / '*.peaks'))]
configs = peakfls2configs(val_files)

In [8]:
IRIS_CHIPSEQ_CONFIG = {
    "mx_dist": 300,
    "peak_size": 301,
    "shades_per_peak": 1,
    "negative_ratio": 100,
    "genome": Path("/home_local/dpenzar/hg38")
}

In [9]:
protocol = ChIPSeqIrisProtocol.from_dt(IRIS_CHIPSEQ_CONFIG)

In [ ]:
#ChIPSeqIrisProtocol.process()

In [15]:
bench = BenchmarkConfig.from_json(BENCHMARK_CONFIG).make_benchmark()

In [16]:
with open(MODELS_PATH, "r") as inp:
    models = json.load(inp)
print(len(models))
for mcfg in models:
    tf_name = mcfg['tf']
    name = mcfg['name']
    path = mcfg["path"]
    bench.add_pfm(tf_name, name, path)

6664


In [21]:
ds = ChIPSeqDataset.load('.ChIPSeqDatasets2/THC_0211_foreign.bed', tp=DatasetType.TRAIN, fmt='.tsv')

In [28]:
executor = ThreadPoolExecutor(max_workers=AVAILABLE_PROCS)

In [29]:
ChIPSeqLOGS.mkdir(exist_ok=True, parents=True)

In [43]:
for cfg in configs[0:10]:
    print(f"Dataset {cfg.name}")
    ds_path = DS_DIR/f"{protocol.get_shades_ds_name(cfg)}.bed"
    ds = ChIPSeqDataset.load(ds_path, tp=DatasetType.TRAIN, fmt='.tsv')
    ds.name = protocol.get_shades_ds_name(cfg)
    ds.tf_name = cfg.tf
    
    futures = {}
    for model in bench.models:
        if not ds.tf_name in model.tfs: # type: ignore
            continue
        ft = executor.submit(bench.score_model_on_ds, model, ds)
        futures[ft] = f"{model.name}@{ds.name}"

    print(f"Submited {len(futures)} futures")

    for ft in as_completed(futures):
        name = futures[ft]
        path = ChIPSeqLOGS / f"{name}.json"
        try:
            scores = ft.result()
            with path.open("w") as outp:
                json.dump(scores, outp)
        except Exception as exc:
            print(exc)
            print(f"Error for {name}")
        else:
            print(f"Finished for {name}")

    

    ds_path = DS_DIR/f"{protocol.get_foreign_ds_name(cfg)}.bed"
    ds = ChIPSeqDataset.load(ds_path, tp=DatasetType.TRAIN, fmt='.tsv')
    ds.name = protocol.get_foreign_ds_name(cfg)
    ds.tf_name = cfg.tf

    futures = {}
    for model in bench.models:
        if not ds.tf_name in model.tfs: # type: ignore
            continue
        ft = executor.submit(bench.score_model_on_ds, model, ds)
        futures[ft] = f"{model.name}@{ds.name}"

    print(f"Submited {len(futures)} futures")
    for ft in as_completed(futures):
        name = futures[ft]
        path = ChIPSeqLOGS / f"{name}.json"

        try:
            scores = ft.result()
            with path.open("w") as outp:
                json.dump(scores, outp)
        except Exception as exc:
            print(f"Error for {name}")
            print(exc)
        else:
            print(f"Finished for {name}")
    


    

Dataset SI0159
Submited 0 futures
Submited 0 futures
Dataset THC_0074
Submited 0 futures
Submited 0 futures
Dataset THC_0246
Submited 90 futures


/home_local/dpenzar/miniconda3/lib/python3.9/site-packages/rpy2/rinterface.py:955: UserWarning: R is not initialized by the main thread.
                Its taking over SIGINT cannot be reversed here, and as a
                consequence the embedded R cannot be interrupted with Ctrl-C.
                Consider (re)setting the signal handler of your choice from
                the main thread.
  warnings.warn(


Finished for CHAMP1.DBD@PBM.HK@wiggy-russet-beetle@Halle.Dimont@Motif_1_astrained.ppm_sumscore@THC_0246_shades
Finished for CHAMP1.DBD@PBM.HK@wiggy-russet-beetle@Halle.Dimont@Motif_1_w0_01_bg4_imw15_astrained.ppm_sumscore@THC_0246_shades
Finished for CHAMP1.DBD@PBM.ME@dorky-beige-nightingale@Halle.Dimont@Motif_1_w0_01_bg4_imw15_astrained.ppm_sumscore@THC_0246_shades
Finished for CHAMP1.FL@CHS@silly-champagne-burmese@HughesLab.Homer@Motif1.ppm_sumscore@THC_0246_shades
Finished for CHAMP1.DBD@SMS@flaky-saffron-walrus@Halle.Dimont@Motif_1_1_posneg_astrained.ppm_sumscore@THC_0246_shades
Finished for CHAMP1.FL@CHS@silly-champagne-burmese@Halle.Dimont@Motif_1_w20_astrained.ppm_best@THC_0246_shades
Finished for CHAMP1.DBD@PBM.HK@wiggy-russet-beetle@Halle.Dimont@Motif_1_astrained.ppm_best@THC_0246_shades
Finished for CHAMP1.FL@CHS@silly-champagne-burmese@HughesLab.Homer@Motif4.ppm_sumscore@THC_0246_shades
Finished for CHAMP1.FL@CHS@silly-champagne-burmese@HughesLab.Homer@Motif2.ppm_best@THC_02

KeyboardInterrupt: 

In [37]:
ds.tf_name

'ZNF454'

In [38]:
for model in bench.models:
        #if not (isinstance(model, Model) or ds.tf_name is None or model.tfs is None or ds.tf_name in model.tfs):
    if not ds.tf_name in model.tfs:
        continue
    print(model)

In [41]:
ds.tf_name

'ZNF454'

In [40]:
model.tfs

['ZNF703']

In [ ]:
from abc import ABCMeta, abstractmethod

from exceptions import WrongDatasetTypeException
from typing import Iterator

@define
class UDataset(metaclass=ABCMeta):
    name: str
    type: DatasetType
    tf_name: str
    entries: Optional[List[SeqEntry]] = field(repr=False)
    metainfo: dict
    disk_path: Path  

    SEQUENCE_FIELD: ClassVar[str] = "sequence"
    LABEL_FIELD: ClassVar[str] = "label"
    TAG_FIELD: ClassVar[str] = "tag"
    NO_INFO_VALUE: ClassVar[str] = "NoInfo"


    def closed(self) -> bool:
        return self.entries is None

    def __len__(self) -> int:
        if self.closed():
            raise Exception("Operation on closed dataset")
        return len(self.entries) # type: ignore

    def __iter__(self) -> Iterator[SeqEntry]:
        if self.closed():
            raise Exception("Operation on closed dataset")
        return iter(self.entries) # type: ignore

    def __getitem__(self, ind: int) -> SeqEntry:
        if self.closed():
            raise Exception("Operation on closed dataset")
        return self.entries[ind] # type: ignore
    
    def get_fields(self) -> list[str]:
        if self.type is DatasetType.TRAIN:
            return self.get_train_fields()
        if self.type is DatasetType.TEST:
            return self.get_test_fields()
        if self.type is DatasetType.VALIDATION:
            return self.get_valid_fields()
        if self.type is DatasetType.FULL:
            return self.get_full_fields()
        raise WrongDatasetTypeException(f"{self.type}")

    @abstractmethod
    def get_train_fields(self):
        raise NotImplementedError()
    
    @abstractmethod
    def get_test_fields(self):
        raise NotImplementedError()

    def get_valid_fields(self):
        return self.get_train_fields()
    
    def get_full_fields(self):
        return self.get_train_fields()

    def get(self, key, default):
        try:
            val = getattr(self, key)
        except AttributeError:
            val = self.metainfo.get(key, default)
        return val
    
    def to_tsv(self,
               path: Path,
               ds_fields: Optional[List[str]] = None):
        fields = self.get_fields()
        
        if ds_fields:
           ds_values = [self.get(fl, self.NO_INFO_VALUE) for fl in ds_fields]

        with open(path, "w") as out:
            if ds_fields:
                header = "\t".join(fields + ds_fields)
            else:
                header = "\t".join(fields)
            print(header, file=out)
            for en in self.entries:
                values = [en.get(fld, self.NO_INFO_VALUE) for fld in fields]  
                if ds_fields:
                    values.extend(ds_values) # type: ignore
                print("\t".join(values), file=out)
                    
    def to_json(self, path, mode: Optional[DatasetType] = DatasetType.TEST):
        raise NotImplementedError()

    @abstractmethod
    def to_canonical_format(self, path, mode: Optional[DatasetType] = DatasetType.TEST):
        raise NotImplementedError()

    def split(self):
        raise NotImplementedError()

    @classmethod
    def load(cls, path: Union[Path, str]) -> 'Dataset':
        raise NotImplementedError()
        